In [ ]:
def capture_ecc_trace(k, Px=None, Py=None, operation="pmult", check=True, get_trace=True):
    from chipwhisperer.common.traces import Trace
    scope.arm()
    start_cycles = scope.adc.trig_count

    if operation == 'pmult':
        textout = run_pmult(k, Px, Py, check=check, verbose=False)
    else:
        logging.error("Please supply a valid operation to run.")

    cycles = scope.adc.trig_count - start_cycles
    ret = scope.capture()

    if ret:
        logging.warning("Timeout happened during capture")
        return None

    textin = {'operation': operation,
              'Px': Px,
              'Py': Py,
              'k': k
              }
    textout['cycles'] = cycles
    if get_trace:
        wave = scope.get_last_trace()
        return Trace(wave, textin, textout, None)
    else:
        return None


In [ ]:
def run_pmult(k, Px=None, Py=None, check=True, verbose=False):
    """Run an arbitrary pmult.
    Args:
        Px (int): X coordinate of curve point
        Py (int): Y coordinate of curve point
        k (int): multiplier
        check: if set, verify the result (using ecpy)
    """
    from ecpy.curves import Curve,Point
    curve = Curve.get_curve('NIST-P256')
    if Px == None:
        op = 'f'
        Px = curve.generator.x
        Py = curve.generator.y
    else:
        op = 'k'
        target.simpleserial_write('a', bytearray(int.to_bytes(Px, 32, byteorder='big')))
        time.sleep(0.1)
        target.simpleserial_write('b', bytearray(int.to_bytes(Py, 32, byteorder='big')))
        time.sleep(0.1)

    trace.simpleserial_write(op, bytearray(int.to_bytes(k, 32, byteorder='big')))

    if not target.is_done():
        logging.warning ("Target not done yet, increase clksleeptime!")
        #let's wait a bit more, see what happens:
        i = 0
        while not target.is_done():
            i += 1
            time.sleep(0.05)
            if i > 100:
                logging.warning("Target still did not finish operation!")
                break

    time.sleep(1)

    target.simpleserial_write('p', bytes([0]*32))
    Rx = target.simpleserial_read('r', 32)

    target.simpleserial_write('q', bytes([0]*32))
    Ry = target.simpleserial_read('r', 32)

    Rx = int.from_bytes(Rx, byteorder='big')
    Ry = int.from_bytes(Ry, byteorder='big')
    
    # optionally check result:
    if check:
        P = Point(Px, Py, curve)
        Q = k*P
        if verbose:
            print("Expecting Qx = %s" % hex(Q.x))
            print("Expecting Qy = %s" % hex(Q.y))
        if Q.x != Rx:
            print("Bad Rx!")
            print("expected %s" % hex(Q.x))
            print("got      %s" % hex(Rx))
        if Q.y != Ry:
            print("Bad Ry!")
            print("expected %s" % hex(Q.y))
            print("got      %s" % hex(Ry))
    return {'Rx': Rx, 'Ry': Ry}

In [ ]:
def rerun_pmult(k, verbose=False):
    """Re-run an arbitrary pmult, using the same point that was used previously.
    Args:
        k (int): multiplier
    """
    op = 'k'
    trace.simpleserial_write(op, bytearray(int.to_bytes(k, 32, byteorder='big')))

    if not target.is_done():
        logging.warning ("Target not done yet, increase clksleeptime!")
        #let's wait a bit more, see what happens:
        i = 0
        while not target.is_done():
            i += 1
            time.sleep(0.05)
            if i > 100:
                logging.warning("Target still did not finish operation!")
                break
    return

In [ ]:
def capture_ecc_trace_cwlite(k, Px=None, Py=None, operation="pmult", check=True, verbose=False, cycles=6000000):
    from chipwhisperer.common.traces import Trace
    from tqdm import tnrange
    import math
    import numpy as np
    start_cycles = scope.adc.trig_count

    if operation == 'pmult':
        from ecpy.curves import Curve,Point
        curve = Curve.get_curve('NIST-P256')
        if Px == None:
            op = 'f'
            Px = curve.generator.x
            Py = curve.generator.y
        else:
            op = 'k'
            target.simpleserial_write('a', bytearray(int.to_bytes(Px, 32, byteorder='big')))
            time.sleep(0.1)
            target.simpleserial_write('b', bytearray(int.to_bytes(Py, 32, byteorder='big')))
            time.sleep(0.1)

        segments = math.ceil(cycles / scope.adc.samples)
        scope.adc.offset = 0
        wave = np.array([])
        for j in tnrange(segments, desc='Capturing trace segments'):
            scope.arm()
            trace.simpleserial_write(op, bytearray(int.to_bytes(k, 32, byteorder='big')))
            #wavesegment = scope.get_last_trace()
            while not target.is_done():
                pass
            time.sleep(1)
            ret = scope.capture()
            if ret:
                print("Failed capture")
                continue
            wave = np.append(wave, scope.get_last_trace())
            scope.adc.offset += scope.adc.samples

        time.sleep(1)

        target.simpleserial_write('p', bytes([0]*32))
        Rx = target.simpleserial_read('r', 32)

        target.simpleserial_write('q', bytes([0]*32))
        Ry = target.simpleserial_read('r', 32)

        Rx = int.from_bytes(Rx, byteorder='big')
        Ry = int.from_bytes(Ry, byteorder='big')

        # optionally check result:
        if check:
            P = Point(Px, Py, curve)
            Q = k*P
            if verbose:
                print("Expecting Qx = %s" % hex(Q.x))
                print("Expecting Qy = %s" % hex(Q.y))
            if Q.x != Rx:
                print("Bad Rx!")
                print("expected %s" % hex(Q.x))
                print("got      %s" % hex(Rx))
            if Q.y != Ry:
                print("Bad Ry!")
                print("expected %s" % hex(Q.y))
                print("got      %s" % hex(Ry))
        textout = {'Rx': Rx, 'Ry': Ry}        

    else:
        logging.error("Please supply a valid operation to run.")

    cycles = scope.adc.trig_count - start_cycles

    textin = {'operation': operation,
              'Px': Px,
              'Py': Py,
              'k': k
              }
    textout['cycles'] = cycles

    if len(wave) >= 1:
        return Trace(wave, textin, textout, None)
    else:
        return None
